In [1]:
import os

In [2]:
with open("./data/questions.txt", "r") as file:
    questions = file.read().split("\n\n")
data = {"category": [], "question": []}
for question in questions:
    question = question.split("\n")
    data["category"].append(question[0])
    data["question"].append(" ".join(question[1:]))
questions_df = pd.DataFrame(data)
questions_df.head()

["NEWSPAPERS\nThe dominant paper in our nation's capital, it's among the top 10 U.S. papers in circulation\nThe Washington Post",
 "OLD YEAR'S RESOLUTIONS\nThe practice of pre-authorizing presidential use of force dates to a 1955 resolution re: this island near mainland China\nTaiwan",
 'NEWSPAPERS\nDaniel Hertzberg & James B. Stewart of this paper shared a 1988 Pulitzer for their stories about insider trading\nThe Wall Street Journal',
 'BROADWAY LYRICS\nSong that says, "you make me smile with my heart; your looks are laughable, unphotographable"\nMy Funny Valentine',
 'POTPOURRI\nIn 2011 bell ringers for this charity started accepting digital donations to its red kettle\nThe Salvation Army|Salvation Army',
 "STATE OF THE ART MUSEUM (Alex: We'll give you the museum. You give us the state.)\nThe Naples Museum of Art\nFlorida",
 '"TIN" MEN\nThis Italian painter depicted the "Adoration of the Golden Calf"\nTintoretto',
 'UCLA CELEBRITY ALUMNI\nThis woman who won consecutive heptathlons a